In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import pystac_client
from odc import stac
import xarray as xr
import rioxarray as rxr

In [ ]:
from dask.distributed import Client
client = Client()  # set up local cluster on the machine
client

### Search data in STAC Catalog

In [ ]:
latitude = 27.163
longitude = 82.608
year = 2023

km2deg = 1.0 / 111
x, y = (longitude, latitude)
r = 1 * km2deg  # radius in degrees
bbox = (x - r, y - r, x + r, y + r)

In [ ]:
catalog = pystac_client.Client.open(
    'https://earth-search.aws.element84.com/v1')

In [ ]:

search = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    bbox=bbox,
    datetime=f'{year}',
    query={
        'eo:cloud_cover': {'lt': 50},
        's2:nodata_pixel_percentage': {'lt': 10}},
    sortby=[
        {'field': 'properties.eo:cloud_cover',
         'direction': 'desc'}
        ]

)
items = search.item_collection()

In [ ]:
# load to xarray
ds = stac.load(
    items,
    bands=['red', 'green', 'blue', 'scl'],
    resolution=10,
    chunks={},  # <-- use Dask
    groupby='solar_day',
    preserve_original_order=True
)

In [ ]:
# Select the first scene
timestamp = pd.to_datetime(items[0].properties['datetime']).tz_convert(None)
scene = ds.sel(time=timestamp)
# Mask nodata values
scene = scene.where(scene != 0)

# Apply scale/offset
scale = 0.0001
offset = -0.1

# Select spectral bands (all except 'scl')
data_bands = [band for band in scene.data_vars if band != 'scl']
for band in data_bands:
  scene[band] = scene[band] * scale + offset


In [ ]:
scene_da = scene.to_array('band')

preview = scene_da.rio.reproject(
    scene_da.rio.crs, resolution=300
)

fig, ax = plt.subplots(1, 1)
fig.set_size_inches(5,5)
preview.sel(band=['red', 'green', 'blue']).plot.imshow(
    ax=ax,
    vmin=0, vmax=0.3)
ax.set_title('RGB Visualization')
ax.set_axis_off()
ax.set_aspect('equal')
plt.show()

In [ ]:
mask = scene.scl.isin([3,8,9,10])

In [ ]:
mask_preview = mask.astype('uint8').rio.reproject(
    mask.rio.crs, resolution=300
)

fig, (ax0, ax1) = plt.subplots(1, 2)
fig.set_size_inches(10,5)
preview.sel(band=['red', 'green', 'blue']).plot.imshow(
    ax=ax0,
    vmin=0, vmax=0.3)
ax0.set_title('RGB Visualization')

# RGBA: Transparent, Red
mask_colormap = ListedColormap(['#00000000', '#FF0000FF'])
mask_preview.plot.imshow(
    ax=ax1,
    cmap=mask_colormap,
    add_colorbar=False)

ax1.set_title('Cloud Mask')
for ax in (ax0, ax1):
  ax.set_axis_off()
  ax.set_aspect('equal')
plt.show()

In [ ]:
scene_masked = scene[data_bands].where(~mask)
scene_masked


In [ ]:
# # 1. Compute NDVI on masked scene
# ndvi = (scene_masked['nir'] - scene_masked['red']) / \
#        (scene_masked['nir'] + scene_masked['red'])

# # 2. Save masked scene to GeoTIFF
# scene_masked.to_array('band').rio.to_raster('masked_scene.tif')

# # 3. Check cloud coverage %
# cloud_pct = float(mask.mean()) * 100
# print(f"Cloud coverage: {cloud_pct:.1f}%")

In [ ]:
fig, ax0 = plt.subplots(1, 1)
fig.set_size_inches(15, 5)

scene_masked_da = scene_masked.to_array('band')
masked_preview = scene_masked_da.rio.reproject(
    scene_masked_da.rio.crs, resolution=300
)
masked_preview.sel(band=['red', 'green', 'blue']).plot.imshow(
    ax=ax0, vmin=0, vmax=0.3)
ax2.set_title('Cloud Removed')

In [ ]:
print(preview.data_vars.variables['red'].shape)
print(preview.data_vars.variables['green'].shape)
print(preview.data_vars.variables['blue'].shape)

In [ ]:
import numpy as np

t = np.stack((preview.data_vars.variables['red'], preview.data_vars.variables['green'], preview.data_vars.variables['blue']), axis=0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

z_coords, y_coords, x_coords = t.nonzero()
ax.scatter(x_coords, y_coords, z_coords, c='red', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

In [ ]:
# preview.coords.variables

### Exercice